<a href="https://colab.research.google.com/github/dinaalshraif/Image-Classification/blob/main/Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'deep-learning-for-msc-coursework-2021:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F27947%2F2289661%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240704%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240704T074457Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da92c8ef1aa475c7a601ae598c00b1bd390a782d7726248335f0b8a3941d00336d28cf73e6a8c57766c4f7202663a50279244b2f4b0ebffa1403fbd36e198ec80016ba91292c47050ab549b59bfee5fee24e3e26c95be9b82a51a19f32224a81b3302274100cfb4f6c674854ed616319ad00b54c08be059bbd569e5bf75347b2a071c664aa464b52d91116eb18f125ee0d8557c3c00fa607ebcb303d328808ffc818ad370530494acba72cfda02a9a1c712f714d1946b6b670006d45fdb48dfca43b87660401cbe7bf0a5837e2a7c6e1d13eeaae9807ec8e1f9782d111fe5e71dc25ab70e3ea6f05f4c9f7d2864d4c175410d10fce22e7b0be00f0cc09482a130,traincsv:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1343296%2F2235745%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240704%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240704T074457Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da4469ce9d278456a9a6c398ca6ed5b17c1c8b0b6107c3bc5c0d7385c7542b2393219a3f1e7f0cd67e6e44b19f167c6da695ed272400237b96a5b8e4de44ed92edad63ec52c259eb901d798828a3ede03450dcc24ccecd6b8892b6d465fe95954d0c952b7afcd0cb632514b1d57377b3331c311bda11542711692a149c7859606ecabfefa13027736e6429842d5d080edf5ee831fb631a7ec7ee656aa1c1203efb0e07049db96b2fbfd69d1f17e1d4654f3bbc96d83bd332110ec2dcee890f658ea55c7d192aa7c8abf832983d1dc2bd7164c68ea13bcaa78252c07cc889c4ed47bd9d7daf9e91998ec114d504a8a6989f4325c70ab6e70158b8f87f31c260203'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# importing the libraries
import pandas as pd
import numpy as np

# for reading and displaying images
from skimage import io, transform
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Load train.csv data
train_data = pd.read_csv("/kaggle/input/traincsv/train.csv")
#test = pd.read_csv('/kaggle/input/deep-learning-for-msc-coursework-2021/test')
sample_submission = pd.read_csv("/kaggle/input/deep-learning-for-msc-coursework-2021/example.csv")
#print(train_data.tail(20))
#print("length of the dataset "+ str(len(train_data)))

# adding path column to indicate each img address
#train_data["path"]=  train_data["Id"].astype(str)
#train_data["Id"]= train_data["path"]+".png"


# data distributions
print(train_data['Type'].value_counts())

# encode the label data to be represent with numbers (0,1,2,3) insteted of  Immune,Connective, Cancer, Normal
train_data['Type'] = pd.factorize(train_data['Type'])[0]

train_data['Type'].value_counts()

<h2> label encoding </h2>
- Connective = 0<br>
- Immune = 1<br>
- Cancer = 2 <br>
- Normal = 3

 <h4>From the result of this code "train_data['Type'].value_counts()" we can see the is imbalance issue amoung data types as shown below:</h4>
 - Immune   =    729 <br>
 - Connective  = 726<br>
 - Cancer     =   546<br>
 - Normal     =   189<br>
 <h4> we need to solve this issue before train the classification model to avoid bias results</h4>

In [ ]:
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

# split into input and output elements
X, y = train_data, train_data['Type']
strategy = {0:729, 1:729, 2:729, 3:729}
y = LabelEncoder().fit_transform(y)
oversample = SMOTE(sampling_strategy=strategy)
X_over, y_over = oversample.fit_resample(X, y)

print((y_over))
X_over = X_over['Id']
y_ov = pd.DataFrame(y_over)

print(y_ov.value_counts())
type(X_over)

In [ ]:
# In this part of code I will use Random Undersampling technique in order to solve imbalancing issue by change
# the classes distribution. Random Undersampling, it is mainly used to randomly delete some records from the
# majority classes (labels)from the training dataset.
# naive resampling, methods because they assume nothing about the data and no heuristics are used.

#from collections import Counter
#from sklearn.datasets import make_classification
#from imblearn.under_sampling import RandomUnderSampler
#from sklearn import datasets


#from sklearn.preprocessing import LabelEncoder
#from imblearn.over_sampling import SMOTE

# define dataset
#X, y = make_classification(n_classes=4, n_samples=2190, n_informative =4 , weights=[0.70, 0.70, 0.70, 0.70], flip_y=0)

# summarize class distribution
#print(Counter(y))
# define undersample strategy
#train_data['Type'].replace({" Connective":0, " Immune":1 , " Cancer":2,  " Normal":3}, inplace=True)
# split into input and output elements
#X, y = train_data, train_data['Type']


#strategy = {0:729, 1:729, 2:729, 3:729}
#y = LabelEncoder().fit_transform(y)
#oversample = SMOTE(sampling_strategy=strategy)
#X_over, y_over = oversample.fit_resample(X, y)

#strategy = {0:189, 1:189, 2:189, 3:189}
#undersample = RandomUnderSampler(sampling_strategy=strategy)
# fit and apply the transform
#X_over, y_over = undersample.fit_resample(X, y)
# summarize class distribution


#print(Counter(y_over))
#X_over = X_over['Id']
#y_ov = pd.DataFrame(y_over)
#y_ov.value_counts()

In [ ]:
# loading training images
import cv2
train_img = []
for img_name in tqdm(X_over):
    # defining the image path
    image_path = '/kaggle/input/deep-learning-for-msc-coursework-2021/train/train/' + str(img_name) + '.png'
    # reading the image
    img = cv2.imread(image_path)
    # normalizing the pixel values
    #img /= 255.0
    # converting the type of pixel to float 32
    img = img.astype('float32')
    # appending the image into the list
    train_img.append(img)

# converting the list to numpy array
train_x = np.array(train_img)
# defining the target
train_y =y_over #train_data['Type'].values
train_x.shape

In [ ]:
# visualizing images
i = 0
plt.figure(figsize=(10,10))
plt.subplot(221), plt.imshow(train_x[i])
plt.subplot(222), plt.imshow(train_x[i+25], cmap='gray')
plt.subplot(223), plt.imshow(train_x[i+50], cmap='gray')
plt.subplot(224), plt.imshow(train_x[i+75], cmap='gray')

In [ ]:
# spiliting the training dataset into 75% train and 25% validation sets

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.25)
c1 = train_x.shape[0]
c2 = val_x.shape[0]
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)


In [ ]:
# converting training images into torch format
train_x = train_x.reshape(c1,3, 64, 64)
train_x  = torch.from_numpy(train_x)

# converting the target into torch format
train_y = train_y.astype(int);
train_y = torch.from_numpy(train_y)

# shape of training data
train_x.shape, train_y.shape

In [ ]:
# converting validation images into torch format
val_x = val_x.reshape(c2,3, 64, 64)
val_x  = torch.from_numpy(val_x)

# converting the target into torch format
val_y = val_y.astype(int);
val_y = torch.from_numpy(val_y)

# shape of validation data
val_x.shape, val_y.shape

In [ ]:
class Net(Module):
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(3, 10, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(10),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv2d(10, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),

        )

        self.linear_layers = Sequential(
            Linear(4 * 16 * 16 , 4)
        )
    # Defining the forward pass
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [ ]:
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

print(model)

In [ ]:
def train(epoch):
    model.train()
    tr_loss = 0
    # getting the training set
    x_train, y_train = Variable(train_x), Variable(train_y)
    # getting the validation set
    x_val, y_val = Variable(val_x), Variable(val_y)
    # converting the data into GPU format
    if torch.cuda.is_available():
        x_train = x_train.cuda()
        y_train = y_train.cuda()
        x_val = x_val.cuda()
        y_val = y_val.cuda()

    # clearing the Gradients of the model parameters
    optimizer.zero_grad()

    # prediction for training and validation set
    output_train = model(x_train)
    output_val = model(x_val)

    # computing the training and validation loss
    loss_train = criterion(output_train, y_train)
    loss_val = criterion(output_val, y_val)
    train_losses.append(loss_train)
    val_losses.append(loss_val)

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)

In [ ]:
# defining the number of epochs
n_epochs = 33
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []
# training the model
for epoch in range(n_epochs):
    train(epoch)

In [ ]:
# plotting the training and validation loss
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend()
plt.show()

In [ ]:
# prediction for training set
with torch.no_grad():
    output = model(train_x.cuda())

softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy on training set
accuracy_score(train_y, predictions)

In [ ]:
# prediction for validation set
with torch.no_grad():
    output = model(val_x.cuda())

softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy on validation set
accuracy_score(val_y, predictions)

In [ ]:
# loading test images
test_img = []

for img_name in tqdm(sample_submission['Id']):
    # defining the image path
    image_path = '/kaggle/input/deep-learning-for-msc-coursework-2021/test/test/' + str(img_name) + '.png'
    # reading the image
    img = cv2.imread(image_path)
    # normalizing the pixel values
    #img /= 255.0
    # converting the type of pixel to float 32
    img = img.astype('float32')
    # appending the image into the list
    test_img.append(img)

# converting the list to numpy array
test_x = np.array(test_img)

test_x.shape

In [ ]:
# converting training images into torch format
test_x = test_x.reshape(400,3, 64, 64)
test_x  = torch.from_numpy(test_x)
test_x.shape

In [ ]:
# generating predictions for test set
with torch.no_grad():
    output = model(test_x.cuda())

softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# replacing the label with prediction
sample_submission['Type'] = predictions
sample_submission['Type'].replace({0 : "Connective", 1: "Immune", 2: "Cancer", 3: "Normal"}, inplace=True)
print(sample_submission.tail(60))
sample_submission['Type'].value_counts()
predictions

In [ ]:
# saving the file
sample_submission.to_csv('submission.csv', index=False)

In [ ]:
# spiliting the training dataset into train and validation sets
from sklearn.model_selection import train_test_split
train, valid_data = train_test_split(train_data, stratify=train_data["Type"], test_size=0.2)

In [ ]:
# encode the label data to be represent with numbers (0,1,2,3) insteted of  Immune,Connective, Cancer, Normal
train_data['Type'] = pd.factorize(train_data['Type'])[0]
train_data

In [ ]:
# data distributions
train_data['Type'].value_counts()

In [ ]:
# creating tensor from train_data
#import torch
targets = torch.tensor(train_data['Type'].values)
targets


In [ ]:
# using the ImageFolder class from torchvision to load the data as PyTorch tensors
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
trainImage_dataset = ImageFolder('/kaggle/input/deep-learning-for-msc-coursework-2021/train',transform=ToTensor())
print(len(trainImage_dataset))
trainImage_dataset[0]